In [9]:
import csv
import random
import numpy as np
import math
import copy

with open("agaricus-lepiota.data", 'r') as file:
    data = list(csv.reader(file))

use_data = random.sample(data, int(len(data)*1/400))
    
trainingData = random.sample(use_data, int(len(use_data)*2/3))
testingData = []
for line in use_data:
    if line not in trainingData:
        testingData.append(line)
    
edible = 0
for l in trainingData:
    print(l)
    if l[0] == "e":
        edible = edible + 1
        
e = get_entropy(edible/len(trainingData))
print(e)

['p', 'x', 's', 'n', 'f', 's', 'f', 'c', 'n', 'b', 't', '?', 's', 'k', 'w', 'w', 'p', 'w', 'o', 'e', 'w', 'v', 'p']
['e', 'f', 'f', 'g', 'f', 'n', 'f', 'w', 'b', 'n', 't', 'e', 'f', 'f', 'w', 'w', 'p', 'w', 'o', 'e', 'k', 'a', 'g']
['e', 'x', 'y', 'y', 't', 'a', 'f', 'c', 'b', 'g', 'e', 'c', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'k', 's', 'g']
['e', 'f', 'y', 'g', 't', 'n', 'f', 'c', 'b', 'u', 't', 'b', 's', 's', 'g', 'g', 'p', 'w', 'o', 'p', 'k', 'v', 'd']
['p', 'x', 'y', 'y', 'f', 'f', 'f', 'c', 'b', 'h', 'e', 'b', 'k', 'k', 'n', 'p', 'p', 'w', 'o', 'l', 'h', 'y', 'p']
['p', 'x', 'y', 'n', 'f', 'y', 'f', 'c', 'n', 'b', 't', '?', 'k', 'k', 'p', 'p', 'p', 'w', 'o', 'e', 'w', 'v', 'l']
['p', 'f', 'y', 'g', 'f', 'f', 'f', 'c', 'b', 'g', 'e', 'b', 'k', 'k', 'p', 'p', 'p', 'w', 'o', 'l', 'h', 'v', 'p']
['p', 'x', 'y', 'n', 't', 'p', 'f', 'c', 'n', 'w', 'e', 'e', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'k', 'v', 'u']
['p', 'f', 'f', 'y', 'f', 'f', 'f', 'c', 'b', 'h', 'e', 'b', 'k', 'k', '

In [2]:
#calculates entropy given one probability
def get_entropy(probability):
    
# get 2nd probability from the first
# we know there are only two probabilities because there are 2 classes
    p1 = probability
    p2 = 1 - p1
    if p1 == 0 or p2 == 0:  #math.log will complain if we give it zero
        entropy = 0         #convention of entropy calculation
    else:
        entropy = -( p1*math.log(p1, 2) + p2*math.log(p2, 2) )    #entropy formula
    return entropy

In [11]:
# calculate gain of a given feature for a given distribution
# feature_values is an array of all the values the feature can take on
def calculate_gain(distribution, entropy_of_dist, feature, feature_values):
    feature_value_counts = np.zeros((3, len(feature_values)), int)

# make feature_value_counts, which is a numpy matrix with num cols = num feature values, num rows = 2
# first row is how many of each feature value is in the distribution, eg feature 2="a" in 10 mushrooms
# second row is how many edible mushrooms there are when feature=faeture_value[i]
# third row is how many poisonous mushrooms there are when feature=faeture_value[i]
# all for calculating the entropy for each value of the feature
# this is weird sorry
    for mushroom in distribution:
        i = 0
        for f in range(0, len(feature_values)):
            if mushroom[feature] == feature_values[f]:
                feature_value_counts[0, f] = feature_value_counts[0, f] + 1
                i = f
        if mushroom[0] == "e":
            feature_value_counts[1, i] = feature_value_counts[1, i] + 1
        else:
            feature_value_counts[2, i] = feature_value_counts[2, i] + 1
       
        
# get gain of the feature using formula
# gets entropy for each feature value using feature_value_counts
    gain = entropy_of_dist
    for y in range(0, len(feature_values)):
        entropy_of_feature = get_entropy(feature_value_counts[1, y]/sum(feature_value_counts[1:,y]))
        gain = gain - 1/len(distribution) * (feature_value_counts[0, y]) * entropy_of_feature
    
    return gain

In [4]:
# finds the feature with the highest gain given a distribution(dataset)
# feature_values is a matrix where row i contains the feature values for feature i+1
# its i+1 not i because the first element in every row of the dataset is "p" or "e", the class label
def get_next_feature(distribution, num_features, feature_values):
    edible_count = 0
    for mushroom in distribution:
        if mushroom[0] == "e":
            edible_count = edible_count + 1
            
    entropy = get_entropy(edible_count/len(distribution))
        
    best_gain = 0
    best_feature = 0
    for feature in range(1, num_features+1):
        new_gain = calculate_gain(distribution, entropy, feature, feature_values[feature-1])
        if  new_gain > best_gain:
            best_gain = new_gain
            best_feature = feature
            
    return best_feature

In [5]:
# split the distribution based on the feature chosen in get_next_feature
# feature_values is a list of the values the feature can take on
# returns as many new distributions as there are feature values
# removes the feature we split on from the distribution, since we dont need to check it anymore
def split_dist(distro, feature, feature_values):
    distribution = copy.deepcopy(distro)
    new_distributions = [[]]*len(feature_values)
        
    for mushroom in distribution:
        f = 0
        for y in range(0, len(feature_values)):
            if mushroom[feature] == feature_values[y]:
                f = y
        
        mushroom.pop(feature)
        new_distributions[f].append(mushroom)
        
    return new_distributions

In [6]:
#not done but will create a branch of the tree?
# recursion ¯\_(ツ)_/¯
def create_branch(sofar, distribution, feature_values, num_features):
    
    if len(distribution) == 0:
        return tree(sofar)
    else:
        next_feature = get_next_feature(distribution, num_features, feature_values)
        next_branches = split_dist(distribution, next_feature, feature_values)
    
        feature_values.pop(next_feature)
    
    #do something to create the tree here!!
    #need to remove the last nodes feature values from feature_values before using it
    
        for branch in next_branches:
            create_tree(branch, feature_values, num_features - 1)

In [7]:
# theoretically done but unable to test until create_branch is done
def create_tree(distribution, num_features):
    feature_values = []
    for f in range(num_features):
        l = []
        for m in distribution:
            l.append(m[f+1])
        
        feature_values.append(list(set(l)))
        
    tree = create_branch([], distribution, feature_values, num_features)
    

In [12]:
# testing using the last question in the decision trees tut
# data for this is in coms3.txt
with open("coms3.txt", 'r') as file:
    test = list(csv.reader(file))
    
k = get_next_feature(test, 3, [['b', 'c', 'a'], ['y', 'n'], ['y', 'n']])
print(k)
q = split_dist(test, 3, ['y', 'n'])
print(q[0])

3
[['e', 'a', 'n'], ['p', 'c', 'y'], ['e', 'c', 'n'], ['e', 'b', 'y'], ['p', 'b', 'n'], ['e', 'c', 'y'], ['p', 'a', 'n'], ['e', 'b', 'y']]
